# This Notebook will be for writing codes for basic ML like loss functions, optimization functions, etc.. 

### SVM Loss 

Required:
1. Weight matrix
2. Sample input data
3. bias vector

Methods:
1. Loss calculate:

    a. Non - vectorized
    b. Half - Vectorized
    c. Full - Vectorized

In [80]:
import numpy as np

In [81]:
def Debug(desc, value):
    print "\n" + desc +  "\n"
    print value

In [125]:
def L2_regularize(W):
    W_squared = np.square(W)
    L2_reg = np.sum(W_squared)
    return L2_reg

In [123]:
# Inputs: 
# @y : 1 x N (ground truth labels)
# @wts : C x D (class x feature dim)
# @datapts : => D x N (features x data points)
# @bias : C x 1

# output:
# loss : SVM loss

# TODO: 
# 1. bias can be incorporated inside wts itself by creating one more column vector and corresponding change in datapts.

def svm_loss(wts, datapts, y, bias):
    # TODO: check dim of matrices
    scores = np.add((np.dot(wts, datapts)), np.repeat(bias[:, np.newaxis], y.size, 1));  # bias will be broadcasted
    Debug("Scores: ", scores)
    
    loss_vec = np.zeros(y.size)
    delta = 1.0
    
    ## No vectorization ##
    '''
    # loop for each data point
    for i in range(y.size):
        loss_i = 0;
        correct_label = y[i]
        correct_class_score = scores[correct_label, i]

        # loop for number of classes
        for j in range(scores.shape[0]):
            if(correct_label != j):
                hinge_loss_i_j = max(scores[j, i] - correct_class_score + delta, 0)
                loss_i += hinge_loss_i_j       
        
        loss_vec[i] = loss_i
    '''
    
    ## Half - vectorized  ##
    '''
    for i in range(y.size):
        loss_i = 0;
        correct_class_score = scores[y[i], i]
                
        loss_margins = np.maximum(0, scores[:, i] - correct_class_score + delta)
        loss_margins[y[i]] = 0   # for correct class margin is 0
        
        loss_i = np.sum(loss_margins)        
        
        loss_vec[i] = loss_i
    '''
    
    ##  FULL VECTORIZED ##
    ''
    #scores_mat = np.repeat(scores[:, np.newaxis], scores.size, axis=1)
    #print "\nScores Repeated: \n"
    #print scores_mat
    
    correct_score_mat = scores[y, np.arange(scores.shape[1])]   # 1 x N
    Debug("Correct Label Scores:", correct_score_mat)
    
    loss_margin = np.maximum(scores - correct_score_mat + delta, 0)
    loss_margin[y, np.arange(loss_margin.shape[1])] = 0     # Making loss of actual labels 0
    Debug("Loss Margin: ", loss_margin)
    
    loss_vec = np.sum(loss_margin, axis=0)
    ''
    
    # Regularize
    reg_rate = 0.5
    loss_vec_reg = np.add(loss_vec, reg_rate * L2_regularize(wts))
    
    full_loss = np.sum(loss_vec_reg) / loss_vec_reg.size
    
    return full_loss

In [112]:
def get_sample_data():
    #wts = np.random.random((3,4))
    wts = np.array([ [  1,   5,   9,   9],
                     [  6,   3,   3,   6],
                     [  7,   5,   8,   9]
                    ], dtype='f')
    
    # TODO: Center the data
    datapts = np.array([[2, 1, 4, 3],
                       [5, 8, 2, 9],
                       [11, 4, 1, 6],
                       [7, 3, 6, 5]
                       ], dtype='f')
    
    y = np.array([1, 0, 2, 1])
    
    bias = np.random.random(wts.shape[0])
    
    return wts, datapts, bias, y

In [113]:
def test_svm_loss():
    wts, datapts, bias, y = get_sample_data()
    Debug("Weights:", wts)
    
    Debug("Data:", datapts)
    
    loss = svm_loss(wts, datapts, y, bias)
    Debug("Loss:", loss)


In [127]:
test_svm_loss()


Weights:

[[ 1.  5.  9.  9.]
 [ 6.  3.  3.  6.]
 [ 7.  5.  8.  9.]]

Data:

[[  2.   1.   4.   3.]
 [  5.   8.   2.   9.]
 [ 11.   4.   1.   6.]
 [  7.   3.   6.   5.]]

Scores: 

[[ 189.5560484   104.5560484    77.5560484   147.5560484 ]
 [ 102.96857559   60.96857559   69.96857559   93.96857559]
 [ 190.36337796  106.36337796  100.36337796  159.36337796]]

Correct Label Scores:

[ 102.96857559  104.5560484   100.36337796   93.96857559]

Loss Margin: 

[[ 87.58747282   0.           0.          54.58747282]
 [  0.           0.           0.           0.        ]
 [ 88.39480238   2.80732956   0.          66.39480238]]

Loss:

323.442969988


In [137]:
# TODO:
# 1. Take care of numerical stability for high scores
# 2. Regularization

def softmax_loss(wts, datapts, y, bias):
    # TODO: check dim of matrices
    scores = np.add((np.dot(wts, datapts)), np.repeat(bias[:, np.newaxis], y.size, 1));  # bias will be broadcasted
    Debug("Scores: ", scores)
    
    correct_scores = scores[y, np.arange(scores.shape[1])]
    Debug("Correct Scores:", correct_scores)

    correct_scores_exp = np.exp(correct_scores)
    
    # TODO: it is better to center the scores for each data point before applying exp and taking sum. 
    # This will avoid numerical bloating
    scores_exp = np.exp(scores)    
    scores_exp_sum = np.sum(scores_exp, axis=0)
    Debug("Exponential Scores Sum:", scores_exp_sum)
    
    prob = correct_scores_exp / scores_exp_sum
    Debug("Softmax output: ", prob)
    
    log_prob = np.log(prob)
    Debug("Log prob: ", log_prob)
    
    loss = -1 * np.sum(log_prob) / y.size
    
    # Regularize
    reg_rate = 0.5
    loss_reg = loss + reg_rate * L2_regularize(wts)
    
    return loss_reg

In [119]:
def test_softmax_loss():
    wts, datapts, bias, y = get_sample_data()
    Debug("Weights:", wts)
    
    Debug("Data:", datapts)
    
    loss = softmax_loss(wts, datapts, y, bias)
    Debug("Loss:", loss)

In [138]:
test_softmax_loss()


Weights:

[[ 1.  5.  9.  9.]
 [ 6.  3.  3.  6.]
 [ 7.  5.  8.  9.]]

Data:

[[  2.   1.   4.   3.]
 [  5.   8.   2.   9.]
 [ 11.   4.   1.   6.]
 [  7.   3.   6.   5.]]

Scores: 

[[ 189.13368041  104.13368041   77.13368041  147.13368041]
 [ 102.24502296   60.24502296   69.24502296   93.24502296]
 [ 190.67718639  106.67718639  100.67718639  159.67718639]]

Correct Scores:

[ 102.24502296  104.13368041  100.67718639   93.24502296]

Exponential Scores Sum:

[  7.83675849e+82   2.30234253e+46   5.29110648e+43   2.22291309e+69]

Softmax output: 

[  3.23826817e-39   7.28639729e-02   1.00000000e+00   1.40888795e-29]

Log prob: 

[ -8.86257800e+01  -2.61916096e+00  -5.96139804e-11  -6.64321670e+01]

Loss:

287.919276977
